In [3]:
import struct
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from utils.colmap_utils import (read_cameras_binary,
                                read_points3D_binary, 
                                read_images_binary, 
                                create_transforms_json)
from utils.plot_utils import visualize_point_cloud
from glob import glob
import cv2
from concurrent.futures import ThreadPoolExecutor

from scipy.spatial import KDTree

In [4]:
# cameras_file = "data/sparse/0/cameras.bin"
# cameras_data = read_cameras_binary(cameras_file)

# if cameras_data:
#     first_camera_id = list(cameras_data.keys())[0]
#     print(f"Camera {first_camera_id} info:")
#     print(cameras_data[first_camera_id])

In [5]:
# points3D_file = "data/dense/dense_points3D.bin"
# not noticing significant difference between sparse / dense (from original gaussian-splatting repository)
points3D_file = "data/sparse/0/points3D.bin"
points3D_data = read_points3D_binary(points3D_file)

points = np.array([v["xyz"] for v in points3D_data.values()])
colors = np.array([v["rgb"] / 255.0 for v in points3D_data.values()])

In [6]:
# print(f"Loaded {len(points)} points")

# visualize_point_cloud(points, colors)

In [7]:
class GaussianSplatting:
    def __init__(self, images_path='data/images', points_path='data/sparse/0/points3D.bin'):
        self.M_points = self.get_sfm_points(points_path)
        # self.images, self.height, self.width = self.get_images(images_path)
    
    def get_images(self, images_path):
        images_data = sorted(glob(images_path + '/*.png'))
        with ThreadPoolExecutor() as executor: # faster loading
            images = list(executor.map(cv2.imread, images_data))
        height, width, _ = images[0].shape
        return images, height, width

    def get_sfm_points(self, points_path):
        """
        𝑀 ← SfM Points ⊲ Positions
        """
        points3D_data = read_points3D_binary(points_path)
        points = np.array([v["xyz"] for v in points3D_data.values()])
        colors = np.array([v["rgb"] / 255.0 for v in points3D_data.values()])
        return points, colors
    
    def init_attributes(self):
        """
        𝑆,𝐶, 𝐴 ← InitAttributes() ⊲ Covariances, Colors, Opacities

        https://medium.com/@yianyao1994/gaussian-splatting-part-2-representation-and-initialization-c0a036adf16e
        
        Center: Set to the point cloud locations from SfM.
        Scaling factor: Initialized as isotropic, using the mean distance to the 3-nearest neighbors.
        Rotation: No initial rotation is applied.
        Opacity: Set to 0.1.
        Spherical Harmonics (SH): Inherited from the color information in the point cloud.

        """
        # 3D covariance matrix Σ
        pass

    def init_gaussian_covariance(self, points):
        """
        Explanation
        Sec 4: 
            Directly optimizing covariance matrix Σ can lead to non positive non semi-definite covariance matrices.
            Instead, we factorize Σ = 𝑅𝑆𝑆^𝑇𝑅^𝑇 [7dof], allowing anisotropic covariance and valid covariance matrices.
            For independent optimization, we store S: 3D vector for scaling and quaternion q (normalize to get valid unit quaternion)

            Convert these splats to pixel space
                W viewing transformation or extrinsics
                J Jacobian projective transformation or intrinsics
                Σ' = JW Σ W^TJ^T

        Initialization
        Sec 5.1
            "We estimate the initial covariance matrix as an isotropic Gaussian 
            with axes equal to the mean of the distance to the closest three points."

        Gradient Computation
        Appendix A
            dΣ' / ds = dΣ'/dΣ * dΣ / ds
            dΣ' / dq = dΣ'/dΣ * dΣ / dq

        Simplify Σ' = JW Σ W^TJ^T using U=JW and Σ' being (symmetric) upper left 2x2
        Σ' = U Σ U^T
        """
        # non gradient implementation for now
        # Have to revist complexity of KDTree
        kdtree = KDTree(points)
        distances, _ = kdtree.query(points, k=4)  # k=4 includes the point itself
        
        nearest_distances = distances[:, 1:4]
        mean_distances = nearest_distances.mean(axis=1)
        
        initial_sigma = np.eye(3) * mean_distances[:, np.newaxis, np.newaxis]

        return initial_sigma



In [8]:
model = GaussianSplatting()
sigmas = model.init_gaussian_covariance(points)

In [13]:
def create_ellipsoids_as_one_mesh(points, sigmas, colors, sphere_resolution=5):
    base_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=sphere_resolution)
    base_sphere.compute_vertex_normals()
    
    all_vertices = []
    all_triangles = []
    all_colors = []

    base_vertices = np.asarray(base_sphere.vertices)
    base_triangles = np.asarray(base_sphere.triangles)

    vertex_count = 0
    
    for center, sigma, color in zip(points, sigmas, colors):
        # scale and translate vertices
        scales = np.diagonal(sigma)
        transformed_vertices = base_vertices * scales + center

        # store the transformed vertices
        all_vertices.append(transformed_vertices)
        # store color for each vertex
        all_colors.append(np.tile(color, (len(base_vertices), 1)))
        
        # shift the triangle indices by the current total count of vertices
        shifted_triangles = base_triangles + vertex_count
        all_triangles.append(shifted_triangles)

        vertex_count += len(base_vertices)
    
    # Combine everything
    all_vertices = np.vstack(all_vertices)
    all_triangles = np.vstack(all_triangles)
    all_colors = np.vstack(all_colors)

    big_mesh = o3d.geometry.TriangleMesh()
    big_mesh.vertices = o3d.utility.Vector3dVector(all_vertices)
    big_mesh.triangles = o3d.utility.Vector3iVector(all_triangles)
    big_mesh.vertex_colors = o3d.utility.Vector3dVector(all_colors)
    big_mesh.compute_vertex_normals()

    return big_mesh

# Then visualize:
big_mesh = create_ellipsoids_as_one_mesh(points, sigmas / 4, colors, sphere_resolution=10)
o3d.visualization.draw_geometries([big_mesh])
